In [ ]:
import pickle 
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, Input
import os
import logging
import IQ as IQ

logging.basicConfig(filename='adversarial_results.log',
                    filemode='a',
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

os.makedirs('data', exist_ok=True)

iq = IQ.IQ(Fc=2439810000 + 0.1e4)

def RV_CNN(X_train, y_train, X_test, y_test):
    input_data = Input(shape=(X_train.shape[1], 1))
    x = Conv1D(filters=16, kernel_size=5, activation='relu')(input_data)
    x = Flatten()(x)
    x = Dense(64, activation='relu', kernel_regularizer=regularizers.L2(0.01))(x)
    x = Dropout(0.5)(x)
    output = Dense(y_test.shape[1], activation='softmax')(x)
    model = Model(inputs=input_data, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return model, accuracy

def add_gaussian_noise(X, snr_db):
    snr_linear = 10 ** (snr_db / 10)
    power_signal = np.mean(np.abs(X) ** 2, axis=1, keepdims=True)
    power_noise = power_signal / snr_linear
    noise = np.sqrt(power_noise / 2) * np.random.randn(*X.shape)
    return X + noise

def add_multipath(X, num_paths=3):
    w = np.random.rand(num_paths)
    w /= w.sum()
    mp_signals = [np.roll(X, shift=i, axis=1)*wt for i, wt in enumerate(w)]
    return np.sum(mp_signals, axis=0)

def add_rayleigh_fading(X):
    f = np.sqrt(np.random.exponential(scale=1, size=X.shape))
    return X * f

def fgsm_attack(model, X, y, epsilon=0.01, targeted=False, target_class=None):
    with tf.GradientTape() as tape:
        tape.watch(X)
        predictions = model(X)
        loss = tf.keras.losses.categorical_crossentropy(
            to_categorical(target_class, y.shape[1]) if targeted else y, predictions
        )
    gradient = tape.gradient(loss, X)
    pert = epsilon * tf.sign(gradient)
    return X - pert if targeted else X + pert

def pgd_attack(model, X, y, epsilon=0.01, alpha=0.001, num_steps=40, targeted=False, target_class=None):
    X_adv = X
    for _ in range(num_steps):
        with tf.GradientTape() as tape:
            tape.watch(X_adv)
            predictions = model(X_adv)
            loss = tf.keras.losses.categorical_crossentropy(
                to_categorical(target_class, y.shape[1]) if targeted else y, predictions
            )
        grad = tape.gradient(loss, X_adv)
        pert = alpha * tf.sign(grad)
        X_adv = X_adv - pert if targeted else X_adv + pert
        X_adv = tf.clip_by_value(X_adv, X - epsilon, X + epsilon)
    return X_adv

def cw_attack(model, X, y, confidence=0.0, targeted=False, target_class=None, max_iterations=1000, learning_rate=0.01):
    def loss_fn(logits, labels):
        real = tf.reduce_sum(labels*logits, axis=1)
        other = tf.reduce_max((1 - labels)*logits - labels*1e4, axis=1)
        return tf.maximum(0.0, other - real + confidence) if targeted else tf.maximum(0.0, real - other + confidence)

    perturbation = tf.Variable(tf.zeros_like(X))
    opt = tf.optimizers.Adam(learning_rate)
    tgt_y = to_categorical(target_class, y.shape[1]) if targeted else y

    for _ in range(max_iterations):
        with tf.GradientTape() as tape:
            adv_x = tf.clip_by_value(X + perturbation, 0, 1)
            logits = model(adv_x)
            loss = tf.reduce_mean(loss_fn(logits, tgt_y))
        grad = tape.gradient(loss, perturbation)
        opt.apply_gradients([(grad, perturbation)])
    return tf.clip_by_value(X + perturbation, 0, 1)

def evaluate_performance(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='macro', zero_division=0),
        'f1_score': f1_score(y_true, y_pred, average='macro', zero_division=0)
    }

def process_dataframe(df):
    if 'I' in df.columns and 'Q' in df.columns:
        if df['I'].apply(lambda x: isinstance(x, (list, np.ndarray))).all() and df['Q'].apply(lambda x: isinstance(x, (list, np.ndarray))).all():
            df['I'] = df['I'].apply(lambda x: np.array(x, dtype=np.float64))
            df['Q'] = df['Q'].apply(lambda x: np.array(x, dtype=np.float64))
            df['frame'] = df.apply(lambda row: row['I'] + row['Q'] * 1j, axis=1)
        else:
            raise ValueError()
    else:
        raise KeyError()
    return df

def configCreator(downSampleRate=1, cutoff=1e6):
    return {
        iq.butter: {'Fs': iq.Fs // downSampleRate, "cutoff": cutoff},
        iq.downSample: {'downSampleRate': downSampleRate, "shift": 0},
        iq.demodulate: {'Fs': iq.Fs}
    }

def freqCreator():
    return {
        iq.gradient: {},
        iq.unwrapPhase: {},
        iq.phase: {},
    }

def runForExperiment(df, target, mode, query, configurations):
    if df[target].dtype == object:
        df[target] = df[target].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    if df[target].dtype == object:
        df[target] = df[target].astype(str).astype(int)

    label_encoder = LabelEncoder()
    df[target] = label_encoder.fit_transform(df[target])
    num_classes = len(label_encoder.classes_)

    best_config = None
    best_accuracy = 0
    best_model = None
    best_X_test = None
    best_y_test = None

    for config_name, config_methods in configurations.items():
        try:
            if mode == 'freq':
                temp = iq.apply(methods={**freqCreator(), **config_methods}, frame=df['frame'])
            else:
                temp = iq.apply(methods=config_methods, frame=df['frame'])

            if mode == 'IQSplit':
                temp = temp.apply(lambda x: np.concatenate([np.real(x), np.imag(x)]))
            else:
                temp = temp.apply(lambda x: x[:min(2000, len(x))])

            X_train, X_test, y_train, y_test = train_test_split(temp, df[target], test_size=0.2, random_state=42)
            y_train_enc = to_categorical(y_train, num_classes=num_classes)
            y_test_enc = to_categorical(y_test, num_classes=num_classes)
            X_train = tf.convert_to_tensor(X_train.tolist(), dtype=tf.float32)
            X_test = tf.convert_to_tensor(X_test.tolist(), dtype=tf.float32)
            model, acc = RV_CNN(X_train, y_train_enc, X_test, y_test_enc)

            if acc > best_accuracy:
                best_accuracy = acc
                best_config = config_name
                best_model = model
                best_X_test = X_test
                best_y_test = y_test_enc

        except:
            pass

    if best_model is not None:
        for snr in [10, 20, 30]:
            X_g = add_gaussian_noise(best_X_test.numpy(), snr)
            X_m = add_multipath(best_X_test.numpy())
            X_f = add_rayleigh_fading(best_X_test.numpy())
            y_true = np.argmax(best_y_test, axis=1)

            for pertX in [X_g, X_m, X_f]:
                y_pred = np.argmax(best_model.predict(pertX), axis=1)
                evaluate_performance(y_true, y_pred)

        # Non-targeted
        attack_configurations = [
            {'type': 'fgsm', 'params': {'epsilon': 0.01, 'targeted': False}},
            {'type': 'pgd', 'params': {'epsilon': 0.01, 'alpha': 0.001, 'num_steps': 40, 'targeted': False}},
            {'type': 'cw', 'params': {'confidence': 0.0, 'targeted': False, 'max_iterations': 1000, 'learning_rate': 0.01}},
        ]

        # Targeted: choose a target class (e.g., 0)
        targeted_attack_configurations = [
            {'type': 'fgsm', 'params': {'epsilon': 0.01, 'targeted': True, 'target_class': 0}},
            {'type': 'pgd', 'params': {'epsilon': 0.01, 'alpha': 0.001, 'num_steps': 40, 'targeted': True, 'target_class': 0}},
            {'type': 'cw', 'params': {'confidence': 0.0, 'targeted': True, 'max_iterations': 1000, 'learning_rate': 0.01, 'target_class': 0}},
        ]

        for attack in attack_configurations + targeted_attack_configurations:
            try:
                if attack['type'] == 'fgsm':
                    X_adv = fgsm_attack(best_model, best_X_test, best_y_test, **attack['params'])
                elif attack['type'] == 'pgd':
                    X_adv = pgd_attack(best_model, best_X_test, best_y_test, **attack['params'])
                elif attack['type'] == 'cw':
                    X_adv = cw_attack(best_model, best_X_test, best_y_test, **attack['params'])

                y_pred_adv = np.argmax(best_model.predict(X_adv), axis=1)
                y_true = np.argmax(best_y_test, axis=1)
                metrics = evaluate_performance(y_true, y_pred_adv)
                logging.info(f"Attack: {attack['type']}, Targeted: {attack['params'].get('targeted', False)}, Metrics: {metrics}")
            except:
                pass

queries = {
    'E1': {'pkl_file': 'data/E1.pkl', 'target': 'dvc'},
    'E2': {'pkl_file': 'data/E2.pkl', 'target': 'pos'},
    'E3': {'pkl_file': 'data/E3.pkl', 'target': 'pos'},
    'E4': {'pkl_file': 'data/E4.pkl', 'target': 'dvc'},
    'E5': {'pkl_file': 'data/E5.pkl', 'target': 'pos'},
    'E6': {'pkl_file': 'data/E6.pkl', 'target': 'dvc'},
    'E7': {'pkl_file': 'data/E7.pkl', 'target': 'dvc'},
    'E8': {'pkl_file': 'data/E8.pkl', 'target': 'dvc'},
    'E9': {'pkl_file': 'data/E9.pkl', 'target': 'dvc'},
    'E10': {'pkl_file': 'data/E10.pkl', 'target': 'dvc'},
    'E11': {'pkl_file': 'data/E11.pkl', 'target': 'dvc'},
}

configurations = {
    'butter4MHz_Fs100MHz': configCreator(downSampleRate=1, cutoff=4e6),
    'butter4MHz_Fs10MHz': configCreator(downSampleRate=10, cutoff=4e6),
    'butter2MHz_Fs100MHz': configCreator(downSampleRate=1, cutoff=2e6),
    'butter2MHz_Fs10MHz': configCreator(downSampleRate=10, cutoff=2e6),
    'butter2MHz_Fs5MHz': configCreator(downSampleRate=20, cutoff=2e6),
    'butter1MHz_Fs100MHz': configCreator(downSampleRate=1, cutoff=1e6),
    'butter1MHz_Fs10MHz': configCreator(downSampleRate=10, cutoff=1e6),
    'butter1MHz_Fs5MHz': configCreator(downSampleRate=20, cutoff=1e6),
    'butter1MHz_Fs2.5MHz': configCreator(downSampleRate=40, cutoff=1e6),
}

Experiments = ['freq', 'IQSplit', 'RV_CNN']

if __name__ == "__main__":
    for q, info in queries.items():
        try:
            with open(info['pkl_file'], 'rb') as f:
                df = pickle.load(f)
            df = process_dataframe(df)
            for experiment in Experiments:
                runForExperiment(df, info['target'], experiment, info, configurations)
        except:
            pass


In [ ]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, Input
import os
import logging
import json

# Setup logging
logging.basicConfig(filename='adversarial_results.log',
                    filemode='a',
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

# Create cache directory
cache_dir = 'data'
os.makedirs(cache_dir, exist_ok=True)

# IQ Module
import IQ as IQ

# Initialize IQ
iq = IQ.IQ(Fc=2439810000 + 0.1e4)

def RV_CNN(X_train, y_train, X_test, y_test):
    input_data = Input(shape=(X_train.shape[1], 1))
    x = Conv1D(filters=16, kernel_size=5, activation='relu')(input_data)
    x = Flatten()(x)
    x = Dense(64, activation='relu', kernel_regularizer=regularizers.L2(0.01))(x)
    x = Dropout(0.5)(x)
    output = Dense(y_test.shape[1], activation='softmax')(x)

    model = Model(inputs=input_data, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

    loss, accuracy = model.evaluate(X_test, y_test)
    return model, accuracy

def add_noise(X, noise_level=0.01):
    noise = np.random.normal(0, noise_level, X.shape)
    return X + noise

def fgsm_attack(model, X, y, epsilon=0.01, targeted=False, target_class=None):
    with tf.GradientTape() as tape:
        tape.watch(X)
        predictions = model(X)
        loss = tf.keras.losses.categorical_crossentropy(y, predictions) if not targeted \
               else tf.keras.losses.categorical_crossentropy(to_categorical(target_class, num_classes=y.shape[1]), predictions)
    gradient = tape.gradient(loss, X)
    perturbation = epsilon * tf.sign(gradient)
    return X + perturbation if not targeted else X - perturbation

def pgd_attack(model, X, y, epsilon=0.01, alpha=0.001, num_steps=40, targeted=False, target_class=None):
    X_adv = X
    for _ in range(num_steps):
        with tf.GradientTape() as tape:
            tape.watch(X_adv)
            predictions = model(X_adv)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions) if not targeted \
                   else tf.keras.losses.categorical_crossentropy(to_categorical(target_class, num_classes=y.shape[1]), predictions)
        gradient = tape.gradient(loss, X_adv)
        perturbation = alpha * tf.sign(gradient)
        X_adv = tf.clip_by_value(X_adv + perturbation if not targeted else X_adv - perturbation, X - epsilon, X + epsilon)
    return X_adv

def cw_attack(model, X, y, confidence=0.0, targeted=False, target_class=None, max_iterations=1000, learning_rate=0.01):
    def loss_fn(logits, labels):
        real = tf.reduce_sum(labels * logits, axis=1)
        other = tf.reduce_max((1 - labels) * logits - labels * 1e4, axis=1)
        if targeted:
            return tf.maximum(0.0, other - real + confidence)
        else:
            return tf.maximum(0.0, real - other + confidence)

    perturbation = tf.Variable(tf.zeros_like(X))
    optimizer = tf.optimizers.Adam(learning_rate)

    for _ in range(max_iterations):
        with tf.GradientTape() as tape:
            adv_x = tf.clip_by_value(X + perturbation, 0, 1)
            logits = model(adv_x)
            loss = tf.reduce_mean(loss_fn(logits, y))
        gradients = tape.gradient(loss, perturbation)
        optimizer.apply_gradients([(gradients, perturbation)])

    return tf.clip_by_value(X + perturbation, 0, 1)

def evaluate_performance(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='macro', zero_division=0),
        'f1_score': f1_score(y_true, y_pred, average='macro', zero_division=0)
    }

def process_dataframe(df):
    if 'I' in df.columns and 'Q' in df.columns:
        if df['I'].apply(lambda x: isinstance(x, (list, np.ndarray))).all() and \
           df['Q'].apply(lambda x: isinstance(x, (list, np.ndarray))).all():
            df['I'] = df['I'].apply(lambda x: np.array(x, dtype=np.float64))
            df['Q'] = df['Q'].apply(lambda x: np.array(x, dtype=np.float64))
            df['frame'] = df.apply(lambda row: row['I'] + row['Q'] * 1j, axis=1)
            print("'frame' column added successfully.")
        else:
            raise ValueError("'I' and 'Q' columns must contain lists or arrays.")
    else:
        raise KeyError("'I' and 'Q' columns are missing in the DataFrame.")
    return df

def configCreator(downSampleRate=1, cutoff=1e6):
    return {
        iq.butter: {'Fs': iq.Fs // downSampleRate, "cutoff": cutoff},
        iq.downSample: {'downSampleRate': downSampleRate, "shift": 0},
        iq.demodulate: {'Fs': iq.Fs}
    }

def freqCreator():
    return {
        iq.gradient: {},
        iq.unwrapPhase: {},
        iq.phase: {},
    }

def runForExperiment(df, target, mode, query, configurations):
    if df[target].dtype == object:
        df[target] = df[target].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else int(x))

    label_encoder = LabelEncoder()
    df[target] = label_encoder.fit_transform(df[target])

    best_config = None
    best_accuracy = 0
    best_model = None
    best_X_test = None
    best_y_test = None

    for config_name, config_methods in configurations.items():
        try:
            if mode == 'freq':
                temp = iq.apply(methods={**freqCreator(), **config_methods}, frame=df['frame'])
            else:
                temp = iq.apply(methods=config_methods, frame=df['frame'])

            if mode == 'IQSplit':
                temp = temp.apply(lambda x: np.concatenate([np.real(x), np.imag(x)]))
            else:
                temp = temp.apply(lambda x: x[:min(2000, len(x))])

            X_train, X_test, y_train, y_test = train_test_split(temp, df[target], test_size=0.2, random_state=42)

            y_train_encoded = to_categorical(y_train)
            y_test_encoded = to_categorical(y_test)

            X_train = tf.convert_to_tensor(X_train.tolist(), dtype=tf.float32)
            X_test = tf.convert_to_tensor(X_test.tolist(), dtype=tf.float32)

            model, benign_accuracy = RV_CNN(X_train, y_train_encoded, X_test, y_test_encoded)
            print(f"Configuration: {config_name}, Mode: {mode}, Accuracy: {benign_accuracy}")

            if benign_accuracy > best_accuracy:
                best_accuracy = benign_accuracy
                best_config = config_name
                best_model = model
                best_X_test = X_test
                best_y_test = y_test_encoded

        except Exception as e:
            print(f"Error in configuration {config_name}: {e}")

    if best_model:
        print(f"Best configuration for {mode}: {best_config} with accuracy {best_accuracy}")

        attack_configurations = [
            {'type': 'fgsm', 'params': {'epsilon': 0.01, 'targeted': False}},
            {'type': 'pgd', 'params': {'epsilon': 0.01, 'alpha': 0.001, 'num_steps': 40, 'targeted': False}},
            {'type': 'cw', 'params': {'confidence': 0.0, 'targeted': False, 'max_iterations': 1000, 'learning_rate': 0.01}},
        ]

        for attack in attack_configurations:
            try:
                if attack['type'] == 'fgsm':
                    X_adv = fgsm_attack(best_model, best_X_test, best_y_test, **attack['params'])
                elif attack['type'] == 'pgd':
                    X_adv = pgd_attack(best_model, best_X_test, best_y_test, **attack['params'])
                elif attack['type'] == 'cw':
                    X_adv = cw_attack(best_model, best_X_test, best_y_test, **attack['params'])

                y_pred_adv = np.argmax(best_model.predict(X_adv), axis=1)
                y_true = np.argmax(best_y_test, axis=1)
                metrics = evaluate_performance(y_true, y_pred_adv)

                print(f"Attack: {attack['type']}, Metrics: {metrics}")
                logging.info(f"Attack: {attack['type']}, Metrics: {metrics}")

            except Exception as e:
                print(f"Error during {attack['type']} attack: {e}")

# Queries
queries = {
    'E1': {'pkl_file': 'data/E1.pkl', 'target': 'dvc'},
    'E2': {'pkl_file': 'data/E2.pkl', 'target': 'pos'},
    'E3': {'pkl_file': 'data/E3.pkl', 'target': 'pos'},
    'E4': {'pkl_file': 'data/E4.pkl', 'target': 'dvc'},
    'E5': {'pkl_file': 'data/E5.pkl', 'target': 'pos'},
    'E6': {'pkl_file': 'data/E6.pkl', 'target': 'dvc'},
    'E7': {'pkl_file': 'data/E7.pkl', 'target': 'dvc'},
    'E8': {'pkl_file': 'data/E8.pkl', 'target': 'dvc'},
    'E9': {'pkl_file': 'data/E9.pkl', 'target': 'dvc'},
    'E10': {'pkl_file': 'data/E10.pkl', 'target': 'dvc'},
    'E11': {'pkl_file': 'data/E11.pkl', 'target': 'dvc'},
}

# Configurations
configurations = {
    'butter4MHz_Fs100MHz': configCreator(downSampleRate=1, cutoff=4e6),
    'butter4MHz_Fs10MHz': configCreator(downSampleRate=10, cutoff=4e6),
    'butter2MHz_Fs100MHz': configCreator(downSampleRate=1, cutoff=2e6),
    'butter2MHz_Fs10MHz': configCreator(downSampleRate=10, cutoff=2e6),
    'butter2MHz_Fs5MHz': configCreator(downSampleRate=20, cutoff=2e6),
    'butter1MHz_Fs100MHz': configCreator(downSampleRate=1, cutoff=1e6),
    'butter1MHz_Fs10MHz': configCreator(downSampleRate=10, cutoff=1e6),
    'butter1MHz_Fs5MHz': configCreator(downSampleRate=20, cutoff=1e6),
    'butter1MHz_Fs2.5MHz': configCreator(downSampleRate=40, cutoff=1e6),
}

# Experiments
Experiments = ['freq', 'IQSplit', 'RV_CNN']

for query_name, query_info in queries.items():
    try:
        with open(query_info['pkl_file'], 'rb') as f:
            df = pickle.load(f)

        df = process_dataframe(df)

        for experiment in Experiments:
            runForExperiment(df, query_info['target'], experiment, query_info, configurations)

    except Exception as e:
        print(f"Error processing query {query_name}: {e}")
